<a href="https://colab.research.google.com/github/BATspock/deeplearning/blob/master/pytorch_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install scikit-image

In [ ]:
from collections import OrderedDict
import torch.nn as nn

In [ ]:
def get_training_model(inFeatures = 4, hiddenDim = 8, nbClasses = 3):
  mlpModel = nn.Sequential(OrderedDict([
                                        ("hidden_layer_1", nn.Linear(inFeatures, hiddenDim)),
                                        ("activation_1", nn.ReLU()),
                                        ("output_layer", nn.Linear(hiddenDim, nbClasses))
  ]))

  return mlpModel

In [ ]:
from torch.optim import SGD
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
import torch.nn as nn
import torch

In [ ]:
def next_batch(inputs, targets, batchSize):
	# loop over the dataset
	for i in range(0, inputs.shape[0], batchSize):
		# yield a tuple of the current batched data and labels
		yield (inputs[i:i + batchSize], targets[i:i + batchSize])

In [ ]:
BATCH_SIZE = 64
EPOCHS = 10
LR = 1e-2
# determine the device we will be using for training
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}...".format(DEVICE))

[INFO] training using cpu...


In [ ]:
print("[INFO] preparing data...")
(X, y) = make_blobs(n_samples=1000, n_features=4, centers=3,
	cluster_std=2.5, random_state=95)
# create training and testing splits, and convert them to PyTorch
# tensors
(trainX, testX, trainY, testY) = train_test_split(X, y,	test_size=0.15, random_state=95)
trainX = torch.from_numpy(trainX).float()
testX = torch.from_numpy(testX).float()
trainY = torch.from_numpy(trainY).float()
testY = torch.from_numpy(testY).float()

[INFO] preparing data...


In [ ]:
mlp = get_training_model().to(DEVICE)
print(mlp)
# initialize optimizer and loss function
opt = SGD(mlp.parameters(), lr=LR)
lossFunc = nn.CrossEntropyLoss()

Sequential(
  (hidden_layer_1): Linear(in_features=4, out_features=8, bias=True)
  (activation_1): ReLU()
  (output_layer): Linear(in_features=8, out_features=3, bias=True)
)


In [ ]:
trainTemplate = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"
# loop through the epochs
for epoch in range(0, EPOCHS):
	# initialize tracker variables and set our model to trainable
	print("[INFO] epoch: {}...".format(epoch + 1))
	trainLoss = 0
	trainAcc = 0
	samples = 0
	mlp.train()
	# loop over the current batch of data
	for (batchX, batchY) in next_batch(trainX, trainY, BATCH_SIZE):
		# flash data to the current device, run it through our
		# model, and calculate loss
		(batchX, batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))
		predictions = mlp(batchX)
		loss = lossFunc(predictions, batchY.long())
		# zero the gradients accumulated from the previous steps,
		# perform backpropagation, and update model parameters
		opt.zero_grad()
		loss.backward()
		opt.step()
		# update training loss, accuracy, and the number of samples
		# visited
		trainLoss += loss.item() * batchY.size(0)
		trainAcc += (predictions.max(1)[1] == batchY).sum().item()
		samples += batchY.size(0)
	# display model progress on the current training batch
	trainTemplate = "epoch: {} train loss: {:.3f} train accuracy: {:.3f}"
	print(trainTemplate.format(epoch + 1, (trainLoss / samples),
		(trainAcc / samples)))

[INFO] epoch: 1...
epoch: 1 train loss: 1.153 train accuracy: 0.401
[INFO] epoch: 2...
epoch: 2 train loss: 0.603 train accuracy: 0.767
[INFO] epoch: 3...
epoch: 3 train loss: 0.394 train accuracy: 0.939
[INFO] epoch: 4...
epoch: 4 train loss: 0.281 train accuracy: 0.975
[INFO] epoch: 5...
epoch: 5 train loss: 0.212 train accuracy: 0.980
[INFO] epoch: 6...
epoch: 6 train loss: 0.169 train accuracy: 0.985
[INFO] epoch: 7...
epoch: 7 train loss: 0.142 train accuracy: 0.984
[INFO] epoch: 8...
epoch: 8 train loss: 0.123 train accuracy: 0.985
[INFO] epoch: 9...
epoch: 9 train loss: 0.110 train accuracy: 0.986
[INFO] epoch: 10...
epoch: 10 train loss: 0.100 train accuracy: 0.986


In [ ]:
	# evaluation mode
	testLoss = 0
	testAcc = 0
	samples = 0
	mlp.eval()
	# initialize a no-gradient context
	with torch.no_grad():
		# loop over the current batch of test data
		for (batchX, batchY) in next_batch(testX, testY, BATCH_SIZE):
			# flash the data to the current device
			(batchX, batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))
			# run data through our model and calculate loss
			predictions = mlp(batchX)
			loss = lossFunc(predictions, batchY.long())
			# update test loss, accuracy, and the number of
			# samples visited
			testLoss += loss.item() * batchY.size(0)
			testAcc += (predictions.max(1)[1] == batchY).sum().item()
			samples += batchY.size(0)
		# display model progress on the current test batch
		testTemplate = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"
		print(testTemplate.format(epoch + 1, (testLoss / samples),
			(testAcc / samples)))
		print("")

epoch: 10 test loss: 0.086 test accuracy: 0.993



# PyTorch Microsoft Tutorial

In [2]:
# Tensors 

%matplotlib inline

import torch
import numpy as np

In [3]:
arr = [[1,2], [3,4]]
np_data = torch.from_numpy(np.array(arr))

In [4]:
#torch ones 
ones_tensor = torch.ones_like(np_data)
#torch random vectors
rand_tensor = torch.rand_like(np_data, dtype=torch.float)
#zeros 
zero_tensor = torch.zeros((2,3))
zero_tensor

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [5]:
rand_tensor.shape, rand_tensor.dtype

(torch.Size([2, 2]), torch.float32)

In [4]:
tensor = torch.ones((4,4))
print(tensor[0])
print("First column", tensor[:,0])
print("Last column", tensor[...,-1])
tensor[:,1] = 0 
print(tensor)

tensor([1., 1., 1., 1.])
First column tensor([1., 1., 1., 1.])
Last column tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [13]:
t = torch.cat([tensor, tensor, tensor], dim = 1)
t

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])

In [12]:
t = torch.cat([tensor, tensor, tensor], dim = 0)
t

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

# Arithmetic Operations

In [5]:
#matrix multiplication
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)
y3 = torch.rand_like(tensor)
torch.matmul(tensor, tensor.T, out=y3)

tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])

In [6]:
y1, y2, y3

(tensor([[3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.]]), tensor([[3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.]]), tensor([[3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.]]))

In [8]:
# element wise multiplication

z1 = tensor*tensor
z2 = tensor.mul(tensor)
z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

In [9]:
z1, z2, z3

(tensor([[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]]), tensor([[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]]), tensor([[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]]))